# SQL Assignment: Learning How to Query Your Own Data from Multiple Databases

In this assignment you will be tasked with creating 3 different databases:

1. The entire Raleigh NC Arrest Dataset
2. A query of raw action description being equivalent to an "on-view arrest"
3. One of the officer with the most stops


The purpose of this assignment is to understand how to create multiple databases that you will have to answer several questions from. Many of the skills you've acquired from the SQL lectures will aid you in successfully completing this assignment. Remember to take your time and utilize all the resources you have.


# **Questions Begin Here:**

## **Load in the raleigh arrest dataset as a pandas dataframe first and display the first 5 rows to help you get started**

In [1]:
#Done in class

from google.colab import drive
drive.mount('/content/drive')



import re
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import bokeh.plotting as bp


%cd /content
df = pd.read_csv('/content/nc_raleigh_arrest_data.csv')

df['date'] = pd.to_datetime(df['date'], format='%m/%d/%y')
# to ex



Mounted at /content/drive
/content


<ipython-input-1-c0fa4ce7f232>:19: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/nc_raleigh_arrest_data.csv')


In [2]:

df.head()

,date,county_name,age,race,sex,officer_id,department_name,type,arrest_made,citation_issued,...,contraband_weapons,frisk_performed,search_conducted,search_person,search_vehicle,search_basis,reason_for_frisk,reason_for_search,reason_for_stop,raw_action_description
0,2002-01-01,Wake County,24,black,female,93e6ddfbcf,Raleigh Police Department,vehicular,False,True,...,NaN,False,False,False,False,NaN,NaN,NaN,Stop Light/Sign Violation,Citation Issued
1,2002-01-01,Wake County,64,white,male,c306a43c0b,Raleigh Police Department,vehicular,False,False,...,NaN,False,False,False,False,NaN,NaN,NaN,Other Motor Vehicle Violation,Verbal Warning
2,2002-01-01,Wake County,25,white,male,6673e6ef48,Raleigh Police Department,vehicular,False,True,...,NaN,False,False,False,False,NaN,NaN,NaN,Other Motor Vehicle Violation,Citation Issued
3,2002-01-01,Wake County,51,black,male,f33c2e2cc6,Raleigh Police Department,vehicular,False,False,...,NaN,False,False,False,False,NaN,NaN,NaN,Driving While Impaired,No Action Taken
4,2002-01-01,Wake County,36,white,male,ac1d3732e8,Raleigh Police Department,vehicular,False,True,...,NaN,False,False,False,False,NaN,NaN,NaN,Speed Limit Violation,Citation Issued


In [3]:
#Question 1
#Create the databases


import sqlite3

# was conn, connn, connnn, but those names didn't remind us what these connections were for

raleigh_arrest_data_conn =sqlite3.connect('raleigh_arrest_data.sqlite')

on_view_arrest_conn = sqlite3.connect('on_view_arrest.sqlite')

officer_most_stops_conn = sqlite3.connect('officer_most_stops.sqlite')



In [4]:
on_view_arrest = df['raw_action_description'] == 'On-View Arrest'
on_view_arrest.head()

#We need to filter

0    False
1    False
2    False
3    False
4    False
Name: raw_action_description, dtype: bool

In [5]:
on_view_arrest = df.query('raw_action_description == "On-View Arrest"  ')

on_view_arrest.head(2)

,date,county_name,age,race,sex,officer_id,department_name,type,arrest_made,citation_issued,...,contraband_weapons,frisk_performed,search_conducted,search_person,search_vehicle,search_basis,reason_for_frisk,reason_for_search,reason_for_stop,raw_action_description
17,2002-01-01,Wake County,21,hispanic,male,2d2a5ba7f7,Raleigh Police Department,vehicular,True,False,...,False,False,True,True,True,other,NaN,Erratic/Suspicious Behavior,Driving While Impaired,On-View Arrest
40,2002-01-01,Wake County,34,hispanic,male,b834298c99,Raleigh Police Department,vehicular,True,False,...,False,False,True,True,True,other,NaN,Observation of Suspected Contraband,Driving While Impaired,On-View Arrest


In [6]:
#Another way to check is to filter using name

on_view_arrest = df.query('raw_action_description == "On-View Arrest"  ')

on_view_arrest['raw_action_description'].unique()



array(['On-View Arrest'], dtype=object)

In [7]:
df['raw_action_description'].unique()

array(['Citation Issued', 'Verbal Warning', 'No Action Taken',
       'On-View Arrest', 'Written Warning', nan], dtype=object)

In [8]:
#officer_most_stops. Shows that a specific officer has had more stops than everyone.

df['officer_id'].value_counts()

officer_id
dec4fca319    3906
55f4bb872f    3495
bac41090c4    3392
fe817a22ae    3386
db839dfc32    3353
              ... 
dfe1fc5636       1
8b6f88c1ad       1
5b990b956e       1
7be9e347c8       1
8fb67ed96f       1
Name: count, Length: 662, dtype: int64

In [9]:
officer_most_stops = df.query('officer_id =="fe817a22ae"  ')

officer_most_stops['officer_id'].unique()

array(['fe817a22ae'], dtype=object)

In [10]:
#pushing data frames to specific databases

df.to_sql('raleigh_arrest_data', raleigh_arrest_data_conn, if_exists='replace', index = False)
on_view_arrest.to_sql('on_view_arrest', on_view_arrest_conn, if_exists='replace', index= False)
officer_most_stops.to_sql('officer_most_stops', officer_most_stops_conn, if_exists='replace', index = False)

3386

In [11]:
#Query the data Then repeat this process for the other two

raleigh_arrest = pd.read_sql('''

SELECT *
  FROM raleigh_arrest_data
  LIMIT 5 ''', con = raleigh_arrest_data_conn )

raleigh_arrest

,date,county_name,age,race,sex,officer_id,department_name,type,arrest_made,citation_issued,...,contraband_weapons,frisk_performed,search_conducted,search_person,search_vehicle,search_basis,reason_for_frisk,reason_for_search,reason_for_stop,raw_action_description
0,2002-01-01 00:00:00,Wake County,24,black,female,93e6ddfbcf,Raleigh Police Department,vehicular,0,1,...,None,0,0,0,0,None,None,None,Stop Light/Sign Violation,Citation Issued
1,2002-01-01 00:00:00,Wake County,64,white,male,c306a43c0b,Raleigh Police Department,vehicular,0,0,...,None,0,0,0,0,None,None,None,Other Motor Vehicle Violation,Verbal Warning
2,2002-01-01 00:00:00,Wake County,25,white,male,6673e6ef48,Raleigh Police Department,vehicular,0,1,...,None,0,0,0,0,None,None,None,Other Motor Vehicle Violation,Citation Issued
3,2002-01-01 00:00:00,Wake County,51,black,male,f33c2e2cc6,Raleigh Police Department,vehicular,0,0,...,None,0,0,0,0,None,None,None,Driving While Impaired,No Action Taken
4,2002-01-01 00:00:00,Wake County,36,white,male,ac1d3732e8,Raleigh Police Department,vehicular,0,1,...,None,0,0,0,0,None,None,None,Speed Limit Violation,Citation Issued


In [12]:
on_view = pd.read_sql('''

SELECT *
  FROM on_view_arrest
  LIMIT 5 ''', con = on_view_arrest_conn )

on_view

,date,county_name,age,race,sex,officer_id,department_name,type,arrest_made,citation_issued,...,contraband_weapons,frisk_performed,search_conducted,search_person,search_vehicle,search_basis,reason_for_frisk,reason_for_search,reason_for_stop,raw_action_description
0,2002-01-01 00:00:00,Wake County,21,hispanic,male,2d2a5ba7f7,Raleigh Police Department,vehicular,1,0,...,0,0,1,1,1,other,None,Erratic/Suspicious Behavior,Driving While Impaired,On-View Arrest
1,2002-01-01 00:00:00,Wake County,34,hispanic,male,b834298c99,Raleigh Police Department,vehicular,1,0,...,0,0,1,1,1,other,None,Observation of Suspected Contraband,Driving While Impaired,On-View Arrest
2,2002-01-01 00:00:00,Wake County,22,white,male,0be7ffe115,Raleigh Police Department,vehicular,1,0,...,0,0,1,1,1,other,None,Erratic/Suspicious Behavior,Stop Light/Sign Violation,On-View Arrest
3,2002-01-02 00:00:00,Wake County,29,black,male,8cad55bd8f,Raleigh Police Department,vehicular,1,0,...,0,0,1,1,0,other,None,Other Official Information,Speed Limit Violation,On-View Arrest
4,2002-01-05 00:00:00,Wake County,26,black,male,be969da7fd,Raleigh Police Department,vehicular,1,0,...,0,0,1,1,1,other,None,Other Official Information,Other Motor Vehicle Violation,On-View Arrest


In [13]:
most_stops = pd.read_sql('''

SELECT *
  FROM officer_most_stops
  LIMIT 5 ''', con = officer_most_stops_conn )
most_stops

,date,county_name,age,race,sex,officer_id,department_name,type,arrest_made,citation_issued,...,contraband_weapons,frisk_performed,search_conducted,search_person,search_vehicle,search_basis,reason_for_frisk,reason_for_search,reason_for_stop,raw_action_description
0,2002-01-07 00:00:00,Wake County,61,black,female,fe817a22ae,Raleigh Police Department,vehicular,0,1,...,None,0,0,0,0,None,None,None,Vehicle Regulatory Violation,Citation Issued
1,2002-01-07 00:00:00,Wake County,33,black,female,fe817a22ae,Raleigh Police Department,vehicular,0,1,...,None,0,0,0,0,None,None,None,Stop Light/Sign Violation,Citation Issued
2,2002-01-07 00:00:00,Wake County,27,white,male,fe817a22ae,Raleigh Police Department,vehicular,0,1,...,None,0,0,0,0,None,None,None,Vehicle Equipment Violation,Citation Issued
3,2002-01-07 00:00:00,Wake County,44,black,male,fe817a22ae,Raleigh Police Department,vehicular,0,1,...,None,0,0,0,0,None,None,None,Seat Belt Violation,Citation Issued
4,2002-01-07 00:00:00,Wake County,21,black,male,fe817a22ae,Raleigh Police Department,vehicular,0,1,...,None,0,0,0,0,None,None,None,Vehicle Regulatory Violation,Citation Issued


In [14]:
#Show tables.

import sqlite3


#con = sqlite3.connect('sql_part_2.sqlite')
#conn = sqlite3.connect('sql_part_1.sqlite')
#connn = fill in
#connnn = fill in

database_list = (raleigh_arrest_data_conn, on_view_arrest_conn, officer_most_stops_conn)

for x in database_list:   # For loop for the database list
  cursor = x.cursor()
  cursor.execute("SELECT name from sqlite_master WHERE type='table';")
  print(cursor.fetchall())

  #Show output of all databases and all tables within them



[('raleigh_arrest_data',)]
[('on_view_arrest',)]
[('officer_most_stops',)]


In [ ]:
# Exercise 1:

# Create 2 databases named: on_view_arrest, officer_most_stops

# When finished print and show that all databases have been created successfully (*use lecture notes to help here*)

# Hint: import pandas as pd, sqlite3

In [ ]:
# Exercise 2:

# Load raleigh_arrest_data file  into raleigh_arrest_data (database), then query the first 5 rows

# Use this syntax below to send the dataframe to your SQLite database:

# dataframe_name.to_sql('new_name_of_dataframe', conn, if_exists='append', index = False)

In [ ]:
# Exercise 3:

#Create the last 2 tables based upon the criteria listed in the assignment prompt by creating a new dataframe

# from filtering then sending those results(tables) to the specific database

# Query both tables to show you have done this correctly

In [ ]:
# Exercise 4:

#Using database 2, write a query that returns the following: the top 5 counts of the reason for stops in

# descending order. Be sure to show your output

#  SELECT reason_for_search, COUNT(1) FROM on_view_arrest GROUP BY reason_for_search ORDER BY 1 DESC LIMIT 5 # chage to count asteric

In [16]:


top_5_reasons = pd.read_sql('''

SELECT reason_for_stop, COUNT(2)
 FROM on_view_arrest
 GROUP BY reason_for_search
 ORDER BY 2 DESC LIMIT 5 ''', con = on_view_arrest_conn )



,reason_for_stop,COUNT(2)
0,Speed Limit Violation,2712
1,Driving While Impaired,1393
2,Vehicle Regulatory Violation,795
3,Driving While Impaired,302
4,Stop Light/Sign Violation,279


In [ ]:
# Exercise 5:

# Find the MAX, AVG, MIN  age(s) of all races using the first database. Show your output

#  SELECT race, MAX(age), AVG(age), MIN(age) FROM raleigh_arrest_data GROUP BY race

In [ ]:
race_age = pd.read_sql('''

SELECT race, MAX(age), AVG(age), MIN(age)
  FROM raleigh_arrest_data
  GROUP BY race ''', con = raleigh_arrest_data_conn )

race_age



,race,MAX(age),AVG(age),MIN(age)
0,asian/pacific islander,110,34.208678,10
1,black,110,33.488765,10
2,hispanic,110,30.643220,10
3,other,80,33.455856,16
4,unknown,110,34.092471,10
5,white,110,34.616843,10


In [ ]:
#Exercise 6:

# Using the third database and only taking into account month by itself, which top 3 months had the most stops? Show output

# Can you derive anything from this? I can derive that the officer was busiest in January and April

# SELECT strftime('%m', date) as month, COUNT(1) FROM officer_most_stops GROUP BY month ORDER BY month ASC
# limit it to 3 oder by count then limit to 3

In [18]:
Months_most_stopped = pd.read_sql('''

SELECT strftime('%m', date) as month, COUNT(1)
  FROM officer_most_stops
  GROUP BY month
  ORDER BY month ASC LIMIT 3''', con = officer_most_stops_conn)

In [ ]:
# Exercise 7:

# Using the 2nd database, what percentage of stops resulted in the person being searched, the vehicle being searched and
# an arrest being made? Show output

In [24]:
import sqlite3

 #connect to on_view_arrest data base
arrest_conn =sqlite3.connect("on_view_arrest.sqlite")

#create a cursor object
cursor = arrest_conn.cursor()

cursor.execute("""
    SELECT
        100.0 * SUM(search_person) / COUNT(*) AS percent_person_searched,
        100.0 * SUM(search_vehicle)/ COUNT(*) AS percent_vehicle_searched,
        100.0 * SUM(arrest_made) / COUNT(*) AS percent_arrest_made
    FROM on_view_arrest
""")
 #fetch the result
result = cursor.fetchone()

 #Close the cursor and connection
cursor.close()
arrest_conn.close()

#Print the result
result

(85.10676475865782, 82.18194868114122, 100.0)

In [ ]:
#Exercise 8:

# Using the first database, identify the average ages of women based on each of the 5 unique reasons for being stopped

woman_age_stop = pd.read_sql('''

SELECT reason_for_stop, AVG(age) AS average_age
FROM raleigh_arrest_data
WHERE sex = 'female'
GROUP BY reason_for_stop
Limit 5;
''' , con = raleigh_arrest_data_conn)

woman_age_stop


,reason_for_stop,average_age
0,Checkpoint,32.722222
1,Driving While Impaired,30.353347
2,Investigation,32.409435
3,Other Motor Vehicle Violation,33.136585
4,Safe Movement Violation,34.931352


In [ ]:
# Exercise 9:

# Show output of all databases and all tables within them

In [ ]:
#Show tables.

import sqlite3


#con = sqlite3.connect('sql_part_2.sqlite')
#conn = sqlite3.connect('sql_part_1.sqlite')
#connn = fill in
#connnn = fill in

database_list = (raleigh_arrest_data_conn, on_view_arrest_conn, officer_most_stops_conn)

for x in database_list:   # For loop for the database list
  cursor = x.cursor()
  cursor.execute("SELECT name from sqlite_master WHERE type='table';")
  print(cursor.fetchall())

  #Show output of all databases and all tables within them
cur.fetchall()

[('raleigh_arrest_data',)]
[('on_view_arrest',)]
[('officer_most_stops',)]


NameError: name 'cur' is not defined

In [ ]:
# Test to show case how to perform SQL query using pandas (example)

engineer = pd.read_sql('''

SELECT *
  FROM student as s
  JOIN course as c
    ON s.course_name = c.course_name
 WHERE s.course_name = 'Engineering' ''' , con= conn)

engineer

In [ ]:
#This version has the corrections we resubmitted